# Pre-processing (Complete)

Data Preprocessing: https://www.kaggle.com/khairulislam/data-preprocessing/notebook

## imports

In [1]:
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# metrics are used to find accuracy or error
from sklearn import metrics

## Functions

In [1]:
# 1. Reading Train and test dataset.
# 2. Check if dataset is reversed.
# 3. Drop 'id', and 'attack_cat' columns.
def import_train_test():
    train = pd.read_csv('../Dataset/UNSW_NB15_training-set.csv')
    test = pd.read_csv('../Dataset/UNSW_NB15_testing-set.csv')
    
    # Dropping the columns based on:
    # https://www.kaggle.com/khairulislam/unsw-nb15-feature-importance
    drop_cols = ['attack_cat', 'id'] + ['response_body_len', 'spkts', 'ct_flw_http_mthd', 'trans_depth', 'dwin', 'ct_ftp_cmd', 'is_ftp_login']
    for df in [train, test]:
        for col in drop_cols:
            if col in df.columns:
                print('Dropping: ', col)
                df.drop([col], axis=1, inplace=True)
    
    if train.shape < test.shape:
        # Reversing the dataset
        train, test = test, train
        print("Train and Test sets are reversed, Corrected Shape:")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    else:
        print("The dataset, is already reversed")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    return train, test

In [3]:
def feature_engineer(df):
    # Everything except: 'FIN', 'INT', 'CON', 'REQ', 'RST is renamed 'others'
    df.loc[~df['state'].isin(['FIN', 'INT', 'CON', 'REQ', 'RST']), 'state'] = 'others'
    # Everything except: ''-', 'dns', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3' is renamed 'others'
    df.loc[~df['service'].isin(['-', 'dns', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3']), 'service'] = 'others'
    # Merging 'igmp', 'icmp', 'rtp' into one protocol: 'igmp_icmp_rtp'
    df.loc[df['proto'].isin(['igmp', 'icmp', 'rtp']), 'proto'] = 'igmp_icmp_rtp'
    # Everything except: 'tcp', 'udp' ,'arp', 'ospf', 'igmp_icmp_rtp' is renamed to 'others'
    df.loc[~df['proto'].isin(['tcp', 'udp','arp', 'ospf', 'igmp_icmp_rtp']), 'proto'] = 'others'
    return df

In [4]:
def get_cat_columns(train):
    # Defining an empty list
    categorical = []
    # Iterating through the columns and checking for columns with datatyp "Object"
    for col in train.columns:
        if train[col].dtype == 'object':
            categorical.append(col) # appending "object" columns to categorical
    return categorical

## Pre-processing

In [5]:
# Importing train test by using the function
train, test = import_train_test()

Dropping:  attack_cat
Dropping:  id
Dropping:  response_body_len
Dropping:  spkts
Dropping:  ct_flw_http_mthd
Dropping:  trans_depth
Dropping:  dwin
Dropping:  ct_ftp_cmd
Dropping:  is_ftp_login
Dropping:  attack_cat
Dropping:  id
Dropping:  response_body_len
Dropping:  spkts
Dropping:  ct_flw_http_mthd
Dropping:  trans_depth
Dropping:  dwin
Dropping:  ct_ftp_cmd
Dropping:  is_ftp_login
Train and Test sets are reversed, Corrected Shape:
Train shape:  (175341, 36)
Test shape:  (82332, 36)


In [6]:
# Splitting the dataset into inputs and outputs
x_train, y_train = train.drop(['label'], axis=1), train['label']
x_test, y_test = test.drop(['label'], axis=1), test['label']
# Running the inputs into the feature_engineer function
x_train, x_test = feature_engineer(x_train), feature_engineer(x_test)

In [7]:
# Getting the categorical and non categorical columns
categorical_columns = get_cat_columns(x_train)
non_categorical_columns = [x for x in x_train.columns if x not in categorical_columns]

In [10]:
x_train.head()

,dur,proto,service,state,dpkts,sbytes,dbytes,rate,sttl,dttl,...,dmean,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,0.121478,tcp,-,FIN,4,258,172,74.087490,252,254,...,43,1,0,1,1,1,1,1,1,0
1,0.649902,tcp,-,FIN,38,734,42014,78.473372,62,252,...,1106,43,1,1,1,1,2,1,6,0
2,1.623129,tcp,-,FIN,16,364,13186,14.170161,62,252,...,824,7,1,2,1,1,3,2,6,0
3,1.681642,tcp,ftp,FIN,12,628,770,13.677108,62,252,...,64,1,1,2,1,1,3,2,1,0
4,0.449454,tcp,-,FIN,6,534,268,33.373826,254,252,...,45,43,1,2,2,1,40,2,39,0


In [11]:
# Using standard scaler to normalize data
scaler = StandardScaler()
x_train[non_categorical_columns] = scaler.fit_transform(x_train[non_categorical_columns])
x_test[non_categorical_columns] = scaler.transform(x_test[non_categorical_columns])

In [12]:
x_train

,dur,proto,service,state,dpkts,sbytes,dbytes,rate,sttl,dttl,...,dmean,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,-0.191029,tcp,-,FIN,-0.135769,-0.049134,-0.102726,-0.576371,0.703839,1.578100,...,-0.314240,-0.775991,-1.366486,-0.645013,-0.544736,-0.554373,-0.705529,-0.715714,-0.753074,-0.126508
1,-0.109485,tcp,-,FIN,0.172599,-0.046410,0.188544,-0.576345,-1.141901,1.560002,...,3.800869,3.147666,-0.318711,-0.645013,-0.544736,-0.554373,-0.614256,-0.715714,-0.288257,-0.126508
2,0.040699,tcp,-,FIN,-0.026933,-0.048527,-0.012133,-0.576734,-1.141901,1.560002,...,2.709185,-0.215468,-0.318711,-0.520827,-0.544736,-0.554373,-0.522983,-0.595543,-0.288257,-0.126508
3,0.049729,tcp,ftp,FIN,-0.063212,-0.047016,-0.098563,-0.576737,-1.141901,1.560002,...,-0.232945,-0.775991,-0.318711,-0.520827,-0.544736,-0.554373,-0.522983,-0.595543,-0.753074,-0.126508
4,-0.140417,tcp,-,FIN,-0.117630,-0.047554,-0.102057,-0.576617,0.723268,1.560002,...,-0.306498,3.147666,-0.318711,-0.520827,-0.420468,-0.554373,2.854115,-0.595543,2.779535,-0.126508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,-0.209773,udp,dns,INT,-0.172047,-0.049958,-0.103923,0.094951,0.723268,-0.720406,...,-0.480703,1.372678,0.729064,2.211259,2.313443,1.520470,1.393748,2.048221,1.385084,-0.126508
175337,-0.131728,tcp,-,FIN,-0.099490,-0.047062,-0.101459,-0.576616,0.723268,1.560002,...,-0.310369,-0.775991,-0.318711,-0.645013,-0.544736,-0.554373,-0.614256,-0.715714,-0.753074,-0.126508
175338,-0.209773,udp,dns,INT,-0.172047,-0.049958,-0.103923,0.094951,0.723268,-0.720406,...,-0.480703,0.251634,0.729064,-0.396641,-0.296199,-0.208566,0.389746,-0.475371,0.269523,-0.126508
175339,-0.209773,udp,dns,INT,-0.172047,-0.049958,-0.103923,0.094951,0.723268,-0.720406,...,-0.480703,1.933201,0.729064,2.956374,3.059055,1.693374,1.941386,2.769248,1.942865,-0.126508


In [13]:
# Using get_dummies to make the categorical values usable.
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print("Column mismatch {0}, {1}".format(set(x_train.columns)- set(x_test.columns),  set(x_test.columns)- set(x_train.columns)))
features = list(set(x_train.columns) & set(x_test.columns))

Column mismatch set(), set()


In [14]:
features = list(set(x_train.columns) & set(x_test.columns))

In [15]:
print(f"Number of features {len(features)}")
x_train = x_train[features]
x_test = x_test[features]

Number of features 53


In [16]:
x_train

,ct_dst_src_ltm,dmean,dur,ct_state_ttl,ct_dst_ltm,dtcpb,dbytes,dinpkt,state_CON,state_FIN,...,ct_dst_sport_ltm,service_ssh,service_ftp,state_RST,service_smtp,smean,proto_ospf,synack,state_REQ,state_INT
0,-0.705529,-0.314240,-0.191029,-1.366486,-0.645013,0.911123,-0.102726,-0.080885,0,1,...,-0.554373,0,0,0,0,-0.458048,0,-0.484346,0,0
1,-0.614256,3.800869,-0.109485,-0.318711,-0.645013,1.557251,0.188544,-0.073735,0,1,...,-0.554373,0,0,0,0,-0.414076,0,-0.484346,0,0
2,-0.522983,2.709185,0.040699,-0.318711,-0.520827,1.472854,-0.012133,0.014711,0,1,...,-0.554373,0,0,0,0,-0.443391,0,0.931748,0,0
3,-0.522983,-0.232945,0.049729,-0.318711,-0.520827,0.058025,-0.098563,0.002046,0,1,...,-0.554373,0,1,0,0,-0.414076,0,-0.484346,0,0
4,2.854115,-0.306498,-0.140417,-0.318711,-0.520827,0.744668,-0.102057,-0.012721,0,1,...,-0.554373,0,0,0,0,-0.409190,0,1.154999,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,1.393748,-0.480703,-0.209773,0.729064,2.211259,-0.715569,-0.103923,-0.089370,0,0,...,1.520470,0,0,0,0,-0.389647,0,-0.484346,0,1
175337,-0.614256,-0.310369,-0.131728,-0.318711,-0.645013,1.834729,-0.101459,-0.021513,0,1,...,-0.554373,0,0,0,0,-0.365219,0,0.365776,0,0
175338,0.389746,-0.480703,-0.209773,0.729064,-0.396641,-0.715569,-0.103923,-0.089370,0,0,...,-0.208566,0,0,0,0,-0.389647,0,-0.484346,0,1
175339,1.941386,-0.480703,-0.209773,0.729064,2.956374,-0.715569,-0.103923,-0.089370,0,0,...,1.693374,0,0,0,0,-0.389647,0,-0.484346,0,1


In [17]:
print('X_train Shape: ', x_train.shape)
print('y_train Shape: ', y_train.shape)
print('X_test Shape: ', x_test.shape)
print('y_test Shape: ', y_test.shape)

X_train Shape:  (175341, 53)
y_train Shape:  (175341,)
X_test Shape:  (82332, 53)
y_test Shape:  (82332,)


## Export CSV

In [ ]:
# merge x_train and y_train before exporting to CSV
x_train['label'] = y_train
x_test['label'] = y_test
x_train.to_csv('../Dataset/train_pp4.csv', index=False)
x_test.to_csv('../Dataset/test_pp4.csv', index=False)